# ALGORITMOS DE ORDENAÇÃO $ N \cdot LOG ~N $

### Proposta:

### Alunos:

- Welison Lucas Almeida Regis - 2019.1,
- Lieverton Santos Silva - 2019.1.

# IMPORTS

In [1]:
import pandas as pd
import random
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# OLYMPIC ATHLETE CLASS

In [2]:
class Athlete:
    
    def __init__(self, name, sex, team, NOC, season, city, event, medal):
        self.name = name
        self.sex = sex
        self.team = team
        self.NOC = NOC
        self.season = season
        self.city = city
        self.event = event
        self.medal = medal

# READ ATHLETES FROM CSV

In [3]:
def read_csv(filename = './assets/athlete_events.csv'): 
    """Method to read specific soccer informations from csv file
    
    attributes:
        filename: path to csv file with athletes data.
    """
    
    df = pd.read_csv(filename, usecols=['Name', 'Sex', 'Team', 'NOC', 'Season', 'City', 'Event', 'Medal'])
    
    athletes = []
    for index, row in df.iterrows():
        name = row['Name']
        sex = row['Sex']
        team = row['Team']
        NOC = row['NOC']
        season = row['Season']
        city = row['City']
        event = row['Event']
        medal = row['Medal']
        
        athlete = Athlete(name, sex, team, NOC, season, city, event, medal)
        athletes.append(athlete)
        
    return athletes

In [4]:
# LIST OF ATHLETES OBJECTS
ATHLETES = read_csv()

# DATASET EXAMPLE

In [5]:
def generate_random_athletes(athletes, n = 5):
    
    random_athletes = []
    for i in range(n):
        athlete = random.choices(athletes)
        random_athletes.append(athlete)
    
    return random_athletes

In [6]:
def athletes_info(athletes):
    athletes = generate_random_athletes(athletes, 5)
        
    athletes_table = []
    for athlete in athletes:
        athlete = athlete[0].__dict__.values()
        athletes_table.append(athlete)

    df = pd.DataFrame(
        athletes_table,
        columns=['Nome', 'Sexo', 'Time', 'NOC', 'Temporada', 'Cidade', 'Evento', 'Medalha']
    )

    return df

In [7]:
athletes_info(ATHLETES.copy())

,Nome,Sexo,Time,NOC,Temporada,Cidade,Evento,Medalha
0,"Patrick ""Paddy"" Kavanagh",M,Ireland,IRL,Summer,London,Football Men's Football,NaN
1,Susan Renate Holmes,F,Great Britain,GBR,Winter,Cortina d'Ampezzo,Alpine Skiing Women's Giant Slalom,NaN
2,Miroslav Varga,M,Czech Republic,CZE,Summer,Beijing,"Shooting Men's Small-Bore Rifle, Prone, 50 metres",NaN
3,Luis Calvet Sandoz,M,Spain,ESP,Summer,Antwerpen,"Shooting Men's Small Bore-Rifle, Standing, 50 ...",NaN
4,Per Erik Fabian Ohlsson,M,Sweden,SWE,Summer,Stockholm,"Shooting Men's Military Rifle, Three Positions...",NaN


# VISUALIZING DATA WITH PANDAS AND PLOTLY

In [8]:
ATHLETE_EVENTS_CSV = pd.read_csv('./assets/athlete_events.csv')
NOC_REGIONS_CSV = pd.read_csv('./assets/noc_regions.csv')

### Count number of competitors per olympic

In [9]:
def plot_summer_winter_athletes(olympics_df):
    summer_df = olympics_df[olympics_df['Season'] == 'Summer']
    winter_df = olympics_df[olympics_df['Season'] == 'Winter']

    summer = go.Scatter(
        x = summer_df['Year'],
        y = summer_df['Athletes'],
        name="Jogos de Verão",
        marker=dict(color="Green"),
        mode = "markers+lines"
    )
    winter = go.Scatter(
        x = winter_df['Year'],
        y = winter_df['Athletes'],
        name="Jogos de Inverno",
        marker=dict(color="Red"),
        mode = "markers+lines"
    )

    data = [summer, winter]
    layout = dict(title = 'Competidores por Olimpíadas',
              xaxis = dict(title = 'Ano', showticklabels=True), 
              yaxis = dict(title = 'Número de Competidores'),
              hovermode = 'closest'
             )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='olympic_athletes')

In [10]:
olympic_athletes = ATHLETE_EVENTS_CSV.groupby('Year')['Season'].value_counts()

olympics_df = pd.DataFrame(
    data = {'Athletes': olympic_athletes.values},
    index = olympic_athletes.index
).reset_index()


plot_summer_winter_athletes(olympics_df)

# OLYMPIC GAMES - COUNTRY'S PARTICIPATION

### Draw 3D map with countries partition

In [11]:
def draw_map(dataset, title, colorscale, map_type = 'orthographic'):
    trace = go.Choropleth(
                locations = dataset['Country'],
                locationmode = 'country names',
                z = dataset['Editions'],
                text = dataset['Country'],
                autocolorscale = False,
                reversescale = False,
                colorscale = colorscale,
                marker = dict(
                    line = dict(
                        color = 'rgb(0,0,0)',
                        width = 0.5)
                ),
                colorbar = dict(
                    title = 'Edições',
                    tickprefix = '')
            )

    data = [trace]
    layout = go.Layout(
        title = title,
        geo = dict(
            showframe = True,
            showlakes = False,
            showcoastlines = True,
            projection = dict(
                type = map_type
            )
        )
    )
    
    fig = dict( data=data, layout=layout)
    iplot(fig)

### Merge athletes with regions datasets

In [12]:
olympics_df = ATHLETE_EVENTS_CSV.merge(NOC_REGIONS_CSV)
olympics_df = olympics_df.rename(columns = {'region':'Country'})

countries_participations = olympics_df.groupby(['Country', 'Season'])['Year'].nunique()

olympic_editions = pd.DataFrame(
    data={'Editions': countries_participations.values},
    index=countries_participations.index
).reset_index()

## Participation per country in summer editions

In [13]:
summer = olympic_editions['Season'] == 'Summer'
summer_editions = olympic_editions[summer]

draw_map(summer_editions, 'PARTICIPAÇÃO MUNDIAL NOS JOGOS OLÍMPICOS DE VERÃO', "Reds")

## Participation per country in winter editions

In [14]:
winter = olympic_editions['Season'] == 'Winter'
winter_editions = olympic_editions[winter]

draw_map(winter_editions, 'PARTICIPAÇÃO MUNDIAL NOS JOGOS OLÍMPICOS DE INVERNO', "Blues", 'equirectangular')